In [1]:
!pip install langchain-google-genai tavily-python -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.3/53.3 kB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.3/66.3 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 426.6/426.6 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 489.1/489.1 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 233.7/233.7 kB 12.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires google-auth==2.43.0, but you have google-auth 2.46.0 which is incompatible.


In [4]:
user_query = input("Enter Your Query: ")
from langchain_google_genai import ChatGoogleGenerativeAI
from google.colab import userdata

def web_search(query):
  from tavily import TavilyClient
  from google.colab import userdata
  client = TavilyClient(userdata.get('tavily'))
  response = client.search(
    query=query,
    include_answer = "basic"
)
  return (response.get("answer"))
print(web_search(user_query))
web_results = web_search(user_query)


Enter Your Query: Asia Cup 2025
The 2025 Asia Cup will take place in September 2025, featuring India, Pakistan, Bangladesh, Sri Lanka, Afghanistan, UAE, Oman, and Hong Kong. The tournament will be held in the UAE. India retained the title by defeating Pakistan in the final.


In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel

llm = ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite",google_api_key=userdata.get('GEMINI_API_KEY'))
# ---------------- FACEBOOK PROMPT ----------------
fb_prompt = PromptTemplate(
    template="""
You are a professional Social Media Manager working for a FAANG-level company.

Goal:
Create a high-quality Facebook post based on the provided web results.

Audience:
General professional audience.

Instructions:
- Write in a confident, professional, and approachable tone
- Keep content concise and informative
- Highlight the key takeaway or value
- Avoid emojis and hashtags
- Do not mention AI or content generation
- Ensure clarity and grammatical correctness

Output Format:
- 2–4 short paragraphs
- Clear call-to-action at the end

Web Results:
{web_reults}
""",
    input_variables=["web_reults"]
)

# ---------------- LINKEDIN PROMPT ----------------
linkedin_prompt = PromptTemplate(
    template="""
You are a Senior LinkedIn Content Strategist for a FAANG-level company.

Goal:
Create a professional and insightful LinkedIn post using the provided web results.

Audience:
Industry professionals, founders, engineers, and decision-makers.

Instructions:
- Use a professional, authoritative, and thought-leadership tone
- Start with a strong opening hook
- Clearly explain why this topic matters to professionals
- Use short paragraphs for readability
- Include 3–5 relevant hashtags at the end
- Do not use emojis
- Do not mention AI or content generation

Output Format:
- 3–5 short paragraphs
- One clear professional insight or takeaway
- Hashtags only at the end

Web Results:
{web_reults}
""",
    input_variables=["web_reults"]
)

# ---------------- TWITTER (X) PROMPT ----------------
twitter_prompt = PromptTemplate(
    template="""
You are a professional Twitter (X) content manager for a FAANG-level company.

Goal:
Create a concise, impactful tweet based on the provided web results.

Audience:
Tech-savvy professionals and general audience.

Instructions:
- Keep the tweet under 280 characters
- Use a clear and engaging tone
- Focus on a single key insight
- Use at most 1–2 relevant hashtags
- Avoid emojis
- Do not mention AI or content generation

Output Format:
- Single tweet
- Plain text only

Web Results:
{web_reults}
""",
    input_variables=["web_reults"]
)

# ---------------- CHAINS ----------------
fb_chain = fb_prompt | llm | StrOutputParser()
linkedin_chain = linkedin_prompt | llm | StrOutputParser()
twitter_chain = twitter_prompt | llm | StrOutputParser()
parallel = RunnableParallel({
    "FB_Post": fb_chain,
    "LinkedIn_Post": linkedin_chain,
    "Twitter_Post": twitter_chain
})
posts = parallel.invoke({"web_reults": web_results})
print(posts)

{'FB_Post': 'Get ready for an exciting showcase of cricketing talent! The 2025 Asia Cup is set to take place in September 2025, hosted in the United Arab Emirates.\n\nThis prestigious tournament will bring together top teams including India, Pakistan, Bangladesh, Sri Lanka, Afghanistan, UAE, Oman, and Hong Kong. Following their victory in the previous edition, India will be looking to defend their title against formidable international competition.\n\nMark your calendars for September 2025 and prepare for thrilling matches and unforgettable moments.', 'LinkedIn_Post': 'The upcoming 2025 Asia Cup, slated for September 2025 in the UAE, underscores the enduring significance of strategic sporting events in fostering global engagement. Beyond the thrill of competition, these tournaments serve as powerful platforms for cross-cultural dialogue and economic interaction.\n\nFor industry professionals, particularly those in sectors like media, sponsorship, and hospitality, understanding the dyna